In [108]:
import sys
import os

PROJ_ROOT = os.path.abspath(os.path.join(os.getcwd(), ".."))
if PROJ_ROOT not in sys.path:
    sys.path.insert(0, PROJ_ROOT)

from src.models.random_forest import RandomForestModel
from src.dataset import Dataset

In [136]:
dataset = Dataset()
f_train, f_test, l_train, l_test = dataset.split_data(target_column = 'LUNG_CANCER')

In [170]:
model = RandomForestModel(n_trees = 120)
model.train(f_train, l_train)

print(model.score(f_train, l_train))
print(model.score(f_test, l_test))

0.990909090909091
0.8928571428571429
